# Simplified Kraken Sequence Classification Algorithm

**Name:** Esra Kashaninia

**Student Number:** 402210676

## Introduction

### Overview
Sequence classification is a fundamental task in bioinformatics, pivotal for understanding biological data derived from sequencing technologies. It involves determining the taxonomic origin of sequences extracted from samples, which can range from single cells to complex environments like soil or ocean water. This classification process is crucial for tasks such as identifying pathogens, studying biodiversity, or understanding microbial communities.

### The Kraken Algorithm
Kraken is a bioinformatics algorithm designed for the rapid and accurate classification of metagenomic sequences using exact k-mer matching. The original Kraken algorithm leverages a sophisticated database structure to map short DNA sequences (k-mers) extracted from longer genomic sequences to the lowest common ancestor (LCA) in a taxonomic tree of all genomes that contain them. This mapping allows the algorithm to classify sequences with high precision based on the taxonomic information of their constituent k-mers.

# Database Preparation

## Downloading the `Kraken2`'s `Viral` Database

In [1]:
!wget "https://genome-idx.s3.amazonaws.com/kraken/k2_viral_20240112.tar.gz" -P /content/viral-k2/

--2024-05-15 06:28:34--  https://genome-idx.s3.amazonaws.com/kraken/k2_viral_20240112.tar.gz
Resolving genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)... 54.231.132.105, 52.216.38.73, 52.216.146.59, ...
Connecting to genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)|54.231.132.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 506061528 (483M) [application/x-tar]
Saving to: ‘/content/viral-k2/k2_viral_20240112.tar.gz’

k2_viral_20240112.t 100%[===================>] 482.62M  24.5MB/s    in 16s     

2024-05-15 06:28:51 (29.5 MB/s) - ‘/content/viral-k2/k2_viral_20240112.tar.gz’ saved [506061528/506061528]



## Extracting the Database

In [2]:
!tar xfvz "/content/viral-k2/k2_viral_20240112.tar.gz" -C '/content/viral-k2/'

hash.k2d
opts.k2d
taxo.k2d
seqid2taxid.map
inspect.txt
ktaxonomy.tsv
library_report.tsv
database100mers.kmer_distrib
database150mers.kmer_distrib
database200mers.kmer_distrib
database250mers.kmer_distrib
database300mers.kmer_distrib
database50mers.kmer_distrib
database75mers.kmer_distrib


## Downloading genome sequences

In [3]:
import pandas as pd

k2_viral_df = pd.read_csv('/content/viral-k2/library_report.tsv', sep='\t')
k2_viral_df['accession_id'] = k2_viral_df['Sequence Name'].apply(lambda x: x.split()[0][1:])

k2_viral_df = k2_viral_df.sort_values('accession_id')

print(k2_viral_df.shape)

(18639, 4)


In [4]:
k2_viral_df.head()

,#Library,Sequence Name,URL,accession_id
15700,viral,">AC_000001.1 Ovine adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000001.1
8372,viral,">AC_000002.1 Bovine adenovirus B, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000002.1
1050,viral,">AC_000003.1 Canine adenovirus 1, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000003.1
12390,viral,">AC_000004.1 Duck adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000004.1
5103,viral,">AC_000005.1 Human mastadenovirus A, complete ...",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000005.1


### Downloading `viral.1.1.genomic.fna.gz` from NCBI

In [5]:
!wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.1.genomic.fna.gz -P /content/viral-ncbi/

--2024-05-15 06:29:04--  https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.1.genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 130.14.250.10, 2607:f220:41e:250::10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168654208 (161M) [application/x-gzip]
Saving to: ‘/content/viral-ncbi/viral.1.1.genomic.fna.gz’

viral.1.1.genomic.f 100%[===================>] 160.84M  40.8MB/s    in 4.9s    

2024-05-15 06:29:09 (32.8 MB/s) - ‘/content/viral-ncbi/viral.1.1.genomic.fna.gz’ saved [168654208/168654208]



In [6]:
!gunzip /content/viral-ncbi/viral.1.1.genomic.fna.gz

In [7]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.9 MB/s eta 0:00:00


In [8]:
from Bio import SeqIO

# Path to the extracted FASTA file
fasta_path = '/content/viral-ncbi/viral.1.1.genomic.fna'

# Load the FASTA file using SeqIO.parse
sequences = list(SeqIO.parse(fasta_path, "fasta"))
len(sequences)

18723

Removing sequences which are not present in the `k2 viral` database:

In [9]:
k2_viral_ids = k2_viral_df['accession_id'].to_list()

In [10]:
sequences = [seq_record for seq_record in sequences if seq_record.id in k2_viral_ids]
print(len(sequences))
print(sequences[:10])

18639
[SeqRecord(seq=Seq('AGTAGTGATCTCCCCATAGTAAAAAAACCACAAGACATAACAAATAGTATCAAA...ACT'), id='NC_029549.1', name='NC_029549.1', description='NC_029549.1 High Plains wheat mosaic virus isolate Nebraska segment RNA 2, complete sequence', dbxrefs=[]), SeqRecord(seq=Seq('AGTAGTGATCTCCCAATTTAAAACACAAAACATAACATCGAATATTAGAATATA...ACT'), id='NC_029550.1', name='NC_029550.1', description='NC_029550.1 High Plains wheat mosaic emaravirus segment 3, complete sequence', dbxrefs=[]), SeqRecord(seq=Seq('AGTAGTGATCTCCCCAAATTTAAAACCACAAACCATAACAAACAATATAGAGAA...ACT'), id='NC_029551.1', name='NC_029551.1', description='NC_029551.1 High Plains wheat mosaic virus isolate Nebraska segment RNA 4, complete sequence', dbxrefs=[]), SeqRecord(seq=Seq('AGTAGTGATCTCCCTAAGTAAACCACAAACAATTTACAATCTATTAGAGAATCT...ACT'), id='NC_029552.1', name='NC_029552.1', description='NC_029552.1 High Plains wheat mosaic virus isolate Nebraska segment RNA 5, complete sequence', dbxrefs=[]), SeqRecord(seq=Seq('AGTAGTGATCTCCCAACTTAAA

In [11]:
sequences = sorted(sequences, key=lambda x: x.id)

Adding `seq` column to the `k2_viral_df`:

In [12]:
k2_viral_df['seq'] = [str(seq_record.seq) for seq_record in sequences]

In [13]:
k2_viral_df.head()

,#Library,Sequence Name,URL,accession_id,seq
15700,viral,">AC_000001.1 Ovine adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000001.1,CATCATCAATAATATACGGTGCATTTTGTGCGTGATGACGTATACA...
8372,viral,">AC_000002.1 Bovine adenovirus B, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000002.1,CATCATCAATAATCTACAGTACACTGATGGCAGCGGTCCAACTGCC...
1050,viral,">AC_000003.1 Canine adenovirus 1, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000003.1,CATCATCAATAATATACAGGACAAAGAGGTGTGGCCTAAATGTTGT...
12390,viral,">AC_000004.1 Duck adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000004.1,CTCATGTCATTAATAAGACCATGCAGAAAATGCAAATGAGGCGAAG...
5103,viral,">AC_000005.1 Human mastadenovirus A, complete ...",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000005.1,CCTATCTAATAATATACCTTATACTGGACTAGTGCCAATATTAAAA...


# Downloading the taxonomy tree

In [14]:
!wget ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz
!tar -xzvf taxdump.tar.gz nodes.dmp names.dmp

--2024-05-15 06:29:49--  ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz
           => ‘taxdump.tar.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 130.14.250.10, 2607:f220:41e:250::10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/taxonomy ... done.
==> SIZE taxdump.tar.gz ... 65188445
==> PASV ... done.    ==> RETR taxdump.tar.gz ... done.
Length: 65188445 (62M) (unauthoritative)

taxdump.tar.gz      100%[===================>]  62.17M  34.6MB/s    in 1.8s    

2024-05-15 06:29:52 (34.6 MB/s) - ‘taxdump.tar.gz’ saved [65188445]

names.dmp
nodes.dmp


Use the given `taxonomy` module to load the tree. You can refer to the `https://github.com/frallain/NCBI_taxonomy_tree` to see more details about this module and how to work with:

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [16]:
from gdrive.MyDrive.taxonomy import NcbiTaxonomyTree

taxonomy_tree = NcbiTaxonomyTree(nodes_filename="./nodes.dmp", names_filename="./names.dmp")

## `accession_id` to `tax_id` mapping

As you probably noticed, the taxonomy tree uses `tax_id` to identify the nodes. However, the `k2_viral_df` uses `accession_id` to identify the genomes. We need to create a mapping between these two identifiers.

The cell below may take a while to run!

In [17]:
!wget ftp://ftp.ncbi.nih.gov/pub/taxonomy/accession2taxid/nucl_gb.accession2taxid.gz
!gunzip -f nucl_gb.accession2taxid.gz

--2024-05-15 07:10:16--  ftp://ftp.ncbi.nih.gov/pub/taxonomy/accession2taxid/nucl_gb.accession2taxid.gz
           => ‘nucl_gb.accession2taxid.gz’
Resolving ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)... 130.14.250.12, 130.14.250.13, 2607:f220:41e:250::12, ...
Connecting to ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)|130.14.250.12|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/taxonomy/accession2taxid ... done.
==> SIZE nucl_gb.accession2taxid.gz ... 2409502166
==> PASV ... done.    ==> RETR nucl_gb.accession2taxid.gz ... done.
Length: 2409502166 (2.2G) (unauthoritative)

nucl_gb.accession2t 100%[===================>]   2.24G  42.0MB/s    in 55s     

2024-05-15 07:11:12 (41.9 MB/s) - ‘nucl_gb.accession2taxid.gz’ saved [2409502166]



In [18]:
accession_list = k2_viral_df['accession_id'].to_list()

with open('accession_list.txt', 'w') as file:
    for accession in accession_list:
        file.write(accession + '\n')

The cell below may take a while to run!

In [19]:
!grep -F -f accession_list.txt nucl_gb.accession2taxid > filtered_accession2taxid.txt

In [20]:
from tqdm import tqdm

def parse_accession_to_taxid(filename):
    accession_to_taxid = {}
    with open(filename, 'r') as file:
        for line in tqdm(file, total=18639):
            parts = line.strip().split('\t')
            accession = parts[1]
            taxid = parts[2]
            accession_to_taxid[accession] = taxid
    return accession_to_taxid

accession_tax_mapping = parse_accession_to_taxid('filtered_accession2taxid.txt')

100%|██████████| 18639/18639 [00:00<00:00, 651735.53it/s]


In [21]:
accession_tax_mapping = dict(sorted(accession_tax_mapping.items()))

k2_viral_df['tax_id'] = accession_tax_mapping.values()

In [22]:
k2_viral_df.head()

,#Library,Sequence Name,URL,accession_id,seq,tax_id
15700,viral,">AC_000001.1 Ovine adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000001.1,CATCATCAATAATATACGGTGCATTTTGTGCGTGATGACGTATACA...,114424
8372,viral,">AC_000002.1 Bovine adenovirus B, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000002.1,CATCATCAATAATCTACAGTACACTGATGGCAGCGGTCCAACTGCC...,129950
1050,viral,">AC_000003.1 Canine adenovirus 1, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000003.1,CATCATCAATAATATACAGGACAAAGAGGTGTGGCCTAAATGTTGT...,10512
12390,viral,">AC_000004.1 Duck adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000004.1,CTCATGTCATTAATAAGACCATGCAGAAAATGCAAATGAGGCGAAG...,130328
5103,viral,">AC_000005.1 Human mastadenovirus A, complete ...",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000005.1,CCTATCTAATAATATACCTTATACTGGACTAGTGCCAATATTAAAA...,129875


# SimpleKraken

We strongly recommend you to read the **Kraken** paper before you start implementing the **SimpleKraken** algorithm. You can find the paper [here](https://genomebiology.biomedcentral.com/articles/10.1186/gb-2014-15-3-r46).

## 1. Extracting K-mers

Kraken uses Jellyfish to extract k-mers from the reference database. You can read the exact implementation details in the Kraken paper. In this tutorial, we will use a simplified version of the k-mer extraction process.

In [23]:
from collections import Counter
def extract_kmers(sequence, k):
    # TODO: implement a simple function that extracts all k-mers from the given sequence
    kmers = []
    for i in range(len(sequence) - k + 1):
        kmers.append(sequence[i : i + k])
    return Counter(kmers)

## 2. Building The Database

Calculate the number of appearances of each k-mer in the given set of genomes, and only keep k-mers that appear more than a threshold `t`.


You can use the `Jellyfish` to calculate the number of appearances of the k-mers efficiently:

In [24]:
!apt install jellyfish

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libhts3 libhtscodecs2 libjellyfish-2.0-2
The following NEW packages will be installed:
  jellyfish libhts3 libhtscodecs2 libjellyfish-2.0-2
0 upgraded, 4 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,283 kB of archives.
After this operation, 3,550 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhtscodecs2 amd64 1.1.1-3 [53.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhts3 amd64 1.13+ds-2build1 [390 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjellyfish-2.0-2 amd64 2.3.0-12ubuntu2 [65.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 jellyfish amd64 2.3.0-12ubuntu2 [775 kB]
Fetched 1,283 kB in 4s (312 kB/s)
Selecting previously unselected package libhtscodecs2:amd64.
(Reading database ... 121918 files a

In [25]:
k2_viral_df["seq"] = k2_viral_df["seq"].apply(lambda x: x.strip().upper())
k2_viral_df["tax_id"] = k2_viral_df["tax_id"].apply(lambda x: int(x.strip()))

viral_smaple = k2_viral_df.sample(n=1000, random_state=42)
test_sample = k2_viral_df.sample(n=100, random_state=2021)

with open('viral_smaple.fasta', 'w') as f:
    for index, row in viral_smaple.iterrows():
        f.write(">seq_" + str(index) + "\n" + row['seq'] + "\n")

In [26]:
viral_smaple.to_excel("viral_smaple.xlsx")
test_sample.to_excel("test_sample.xlsx")

In [27]:
del k2_viral_df
del accession_tax_mapping
del accession_list
del sequences

In [28]:
K = 31
mini_len = 7

Run the `jellyfish` with the proper arguments:

In [29]:
!jellyfish count -m 31 -o viral_output.jf -c 3 -s 10000000 -t 50 viral_smaple.fasta

In [30]:
!jellyfish dump -L 10 viral_output.jf > viral_dump.fa

Parse the result file to get a list of suitable k-mers:

In [31]:
with open('viral_dump.fa','r') as f:
    kmers = f.read()

filtered_kmers = []
for i, line in enumerate(kmers.split("\n")):
    if i % 2 == 0:
        try:
            count = int(line[1:].strip())
        except ValueError:
            count = 0
    if i % 2 == 1:
        if count > 10:
            filtered_kmers.append(line.upper())

In [32]:
print(filtered_kmers[:100])
del kmers

['AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA', 'TGAAAGCCAAAATCTATCGAAAATAACTTTA', 'CGTATGTCTGGGCGTGCTAGTCGTGCGTAGC', 'TACTTTTAGATCTAAACTGAAAGCCAAAATC', 'CCGTATGTCTGGGCGTGCTAGTCGTGCGTAG', 'AAGTGCCCGAGCCTGTGCCAAGTTTTCCTAA', 'TGCCCGGTCCTGGAGCCCGGGGCCGGGGGCC', 'CACGACTAGCACGCCCAGACATACGGTAAAA', 'TAAAATTTGACATCAACGATAAATTAATTTG', 'TTTGACATCAACGATAAATTAATTTGGTAGA', 'CAGCAGCAGCAGCAGCAGCAGCAGCAGCAGC', 'GAGCCCCGGCAGCACCCCAGGAGCCCCGGCA', 'CGGGTGCCCCTGGGTCCGCTGCCCCGCTCCG', 'ACCAGGCCGGCCGGAGGGACCCCGGCAGCCC', 'TTTTTTGAAAACTGAAAATTATTTTTTACCG', 'ACTGAAAATTATTTTTTACCGTATGTCTGGG', 'TGTGCCAAGTTTTCCTAACCACTAATGATAA', 'TTTTAGATCTAAACTGAAAGCCAAAATCTAT', 'AATCTATCGAAAATAACTTTACTTTTAGATC', 'AGTGTTTGAGTGTTTGAGTGTTTGAGTGTTT', 'TTTTCAAAAAAAGTGCCCGAGCCTGTGCCAA', 'CCTCCTCCCGCTCCTCCTCCCGCTCCTCCTC', 'CAGCGGCTGCCCAATCGTCGGATATTCAAAA', 'ATATTCAAAAACCCAGCTATCAGTATCCAGC', 'TCCTCATCCTCATCCTCATCCTCATCCTCAT', 'GCCCGGGGCCGGGGGCCGGGTGCCCCTGGGT', 'TTTTGAAAACTGAAAATTATTTTTTACCGTA', 'GCTACGCACGACTAGCACGCCCAGACATACG', 'GTTTTCAAAAAAAGTGCC

Build a database (a `dict`) that assigns each k-mer in the `filtered_kmers` to a list of all genome IDs that have this k-mer:

In [33]:
def extract_kmers(sequence, k):
    # TODO: implement a simple function that extracts all k-mers from the given sequence
    if k >= len(sequence):
        return {sequence: 1}
    kmers = []
    for i in range(len(sequence) - k + 1):
        kmers.append(sequence[i : i + k])
    return Counter(kmers)

In [34]:
def get_minimizer(kmer, mini_len=mini_len):
    mnmzr = kmer[:mini_len]
    for i in range(1, len(kmer) - mini_len + 1):
        mnmzr = min(mnmzr, kmer[i: i + mini_len])
    return mnmzr

In [35]:
from collections import defaultdict


def build_database(sequences, k):
    global viral_sample, filtered_kmers
    database = defaultdict(lambda: defaultdict(list))
    # for km in filtered_kmers:
    #     minimizer = get_minimizer(km)
    # MiaSanMia
    #     database[minimizer][km] = []
    for index, row in tqdm(viral_smaple.iterrows()):
        seq = row["seq"]
        species = row["tax_id"]
        kmers = extract_kmers(seq, k=k)
        if seq is None or len(seq) == 0 or species is None:
            continue
        for km in kmers.keys():
            minimizer = get_minimizer(km)
            database[minimizer][km].append(species)
    return database

## 3. Taxonomy tree and Lowest Common Ancestor (LCA)

Using the database created in the previous step, and the given taxonomy tree, for each k-mer in the genomes, find the LCA of those genomes on the taxonomy tree. Create a dictionary for each internal node in the tree that contains the minimizer of the k-mer and maps it to the k-mer:

$$node_i = \{'minimizer1': 'k-mer'\}$$

In [36]:
db = build_database("", k=K)

1000it [08:07,  2.05it/s]


In [37]:

def lca(taxidlist):
    global taxonomy_tree
    ancestors = taxonomy_tree.getAscendantsWithRanksAndNames(taxidlist, only_std_ranks=False)
    taxids = [[ancestor.taxid for ancestor in ancestors[node]][::-1] for node in taxidlist if not node is None]
    ii = 0
    lowestcommon = 1
    while ii < min(30, min([len(ancestry) for ancestry in taxids])):
        ithdads = [ancestry[ii] for ancestry in taxids]
        if min(ithdads) != max(ithdads):
            return lowestcommon
        lowestcommon = ithdads[0]
        ii += 1

to_delete = []
for minimizer in db.keys():
    for kmer in db[minimizer].keys():
        tax_ids = db[minimizer][kmer]
        if len(tax_ids) > 1:
            db[minimizer][kmer] = [lca(tax_ids)]
        # To save us from None
        if db[minimizer][kmer][0] is None:
            to_delete.append((minimizer, kmer))
for minimizer, kmer in to_delete:
    del db[minimizer][kmer]


## 4. Scoring and Decision Making

To search and score a given genome $g$ through the KDB, we first create the k-mers of the $g$, and then calculate their minimizers and count the number of sequential k-mers $s$ that have that minimizer. Then for each k-mer we search through KDB to find the candidate internal nodes that have the corresponding minimizer in their keys. Then we use exact match through those k-mer values to find the unique internal node and assigns the $s$ to that node. At the end we do a root to leaf traverse and aggregate the scores of parents to calculate leaf scores. The leaf with the highest score is the target.


In [38]:
def find_genome_species(gnm, k):
    global db, taxonomy_tree
    score = defaultdict(int)
    kmers = extract_kmers(gnm, k=k)
    for km in kmers.keys():
        minimizer = get_minimizer(km)
        if len(db[minimizer][km]) > 0:
            score[db[minimizer][km][0]] += kmers[km]
    ancestors = taxonomy_tree.getAscendantsWithRanksAndNames(list(score.keys()), only_std_ranks=False)
    taxids = {node: [ancestor.taxid for ancestor in ancestors[node]][::-1] for node in list(score.keys())}
    for t_id in list(score.keys()):
        for parent in taxids[t_id]:
            score[t_id] += score[parent]
    if len(list(score.keys())) == 0:
        return -1
    max_value = max(score.values())
    candidates = [k for k, v in score.items() if v == max_value]
    if len(candidates) > 1:
        return lca(candidates)
    return candidates[0]

# Evaluation

Generate a few random genomes from the `k2_viral_df` and classify them using the `SimpleKraken` algorithm. Compare the result with the actual taxonomy of the genome.
You can also use a read of a genome and mutate it (e.g. poisson mutation) to see how the algorithm performs in a more realistic scenario:

In [39]:
import numpy as np

correct = 0

alles = test_sample.shape[0]
for index, row in tqdm(test_sample.iterrows()):
        seq = row["seq"]
        species = int(row["tax_id"])

        candidate = find_genome_species(seq, k=K)
        if species == candidate:
            correct += 1

print("Accuracy:", np.round(correct * 100 / alles, 2))

100it [00:35,  2.81it/s]


Accuracy: 5.0


In [40]:
# import json
# with open("db.json","w") as f:
#     json.dump(db, f)

# Bonus

It's worth mentioning that you can use the `kraken2` tool to classify sequences using the database we downloaded in the first section. The `kraken2` tool is a more sophisticated implementation of the Kraken algorithm that uses a database structure optimized for fast and accurate classification of metagenomic sequences.

I'll put a few cells below which will help you install the `kraken2` tool and you can play around with it to classify sequences using the downloaded database.

You can find more information about the `kraken2` tool in the official [GitHub repository](https://github.com/DerrickWood/kraken2/wiki/Manual).

NOTE: The session will be restarted after running the cell below:

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
import condacolab
condacolab.check()

In [ ]:
!conda install bioconda::kraken2

Now you can use the `kraken2`. Have fun!